# Thématique : la Colonisation

## Nous avons télécharger sur CAMille-ULB les journaux du journal "Le soir" avec la requête ("La Colonisation"~4)

## Tous outils à importer

In [9]:
import os
import PyPDF2


# d'autres importation

from collections import Counter
from wordcloud import WordCloud
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from IPython.display import Image

# Le reste des imports

from collections import defaultdict
import spacy
from spacy.lang.fr.examples import sentences
%pip install  fr_core_news_md

[nltk_data] Downloading package stopwords to /home/boy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Note: you may need to restart the kernel to use updated packages.


## Vérifions  d'abord si tous nos fichiers sont en extension TXT

In [11]:

txt_path = '../data/txt'

txts = []
for f in os.listdir(txt_path):
    if os.path.isfile(os.path.join(txt_path, f)):
        txts.append(f)
len(txts)

1000

In [12]:
#  Lister les fichiers pdf qui n'ont pas encore été convertis en txt
not_converted_pdfs = []
for pdf in txts:
    file_name = os.path.splitext(pdf)[0]
    if file_name.endswith(".pdf") :
        not_converted_pdfs.append(pdf)
len(not_converted_pdfs)
print(not_converted_pdfs)

[]
